# Zero-shot classification for job description categories

This applies a zero-shot classification approach to align job postings with the same 24 predefined categories used in the resume dataset. Using the LLaMA-3-70B model via IBM Watsonx, it classifies each job description into one of the resume categories—or assigns “OTHER” if no match is found. The output is a cleaned and consistently labeled job postings dataset, enabling comparability for downstream CV-job matching tasks.

In [ ]:
# Imports

import pandas as pd
from decouple import config

import time
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
from tqdm import tqdm

In [3]:

# Read the CSV file into a DataFrame
job_posts = pd.read_csv('data/jobposts.csv')

In [ ]:
# Define categories for job posts (based on CV categories)
resume_categories = [
     'INFORMATION-TECHNOLOGY',
     'BUSINESS-DEVELOPMENT',
     'FINANCE',
     'ADVOCATE',
     'ACCOUNTANT',
     'ENGINEERING',
     'CHEF',
     'AVIATION',
     'FITNESS',
     'SALES',
     'BANKING',
     'HEALTHCARE',
     'CONSULTANT',
     'CONSTRUCTION',
     'PUBLIC-RELATIONS',
     'HR',
     'DESIGNER',
     'ARTS',
     'TEACHER',
     'APPAREL',
     'DIGITAL-MEDIA',
     'AGRICULTURE',
     'AUTOMOBILE',
     'ВРО',
]

extended_resume_categories = [
    'INFORMATION-TECHNOLOGY',
     'BUSINESS-DEVELOPMENT',
     'FINANCE',
     'ADVOCATE',
     'ACCOUNTANT',
     'ENGINEERING',
     'CHEF',
     'AVIATION',
     'FITNESS',
     'SALES',
     'BANKING',
     'HEALTHCARE',
     'CONSULTANT',
     'CONSTRUCTION',
     'PUBLIC-RELATIONS',
     'HR',
     'DESIGNER',
     'ARTS',
     'TEACHER',
     'APPAREL',
     'DIGITAL-MEDIA',
     'AGRICULTURE',
     'AUTOMOBILE',
     'ВРО',
     'OTHER'
]

In [5]:
WX_API_KEY = config('WX_API_KEY')

credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)

client = APIClient(
    credentials=credentials, 
    project_id="c8d96942-6948-4a06-9909-2ea6394aca25"
)

## Hyperparameters setup

In [6]:
PARAMS = TextGenParameters(
    temperature=0,              # Higher temperature means more randomness - In this case we don't want randomness
    max_new_tokens=10,          # Maximum number of tokens to generate
    stop_sequences=[".", "\n"], # Stop generating text when these sequences are encountered
)

model = ModelInference(
    api_client=client,
    model_id="meta-llama/llama-3-3-70b-instruct",  
    params=PARAMS
)

In [7]:
SYSTEM_PROMPT = """You task is to classify the job descriptions into one of the given categories. If you think the data doesn't fit in any of them assign it to OTHER. Please assign the correct category to the text. Answer with the correct category and nothing else.

CATEGORIES:
{categories}

JOB POST:
{text}

The correct job category for the job post is:
"""

In [ ]:
CATEGORIES = "- " + "\n- ".join(extended_resume_categories)  # Create a string with all categories

predictions = []

refined_df = []


for index, row in tqdm(job_posts.iterrows()):
    try:
        text= (row['job_title'] + ". " + row['job_description'])

        # format the prompt with the categories and the text
        prompt = SYSTEM_PROMPT.format(categories=CATEGORIES, text=text)
        
        # generate the response from the model
        response = model.generate(prompt)

        # extract the generated text from the response
        prediction = response["results"][0]["generated_text"].strip().replace(".", "")

        row['category'] = prediction

        if prediction in resume_categories:
            # add the prediction to the new dataframe
            refined_df.append(row)
        else:
            print(prediction)
    except Exception as e:
        time.sleep(5)
        # By this way we can know about the type of error occurring
        print("The error is: ",e)



18it [00:09,  2.24it/s]

OTHER


31it [00:14,  2.19it/s]

ENGINEERING -> No, it is more specific


65it [00:28,  2.73it/s]

...

OTHER


21959it [2:20:57,  2.91it/s]

SECURITY


22000it [2:21:14,  2.60it/s]

CONTRACT-ADMINISTRATOR


In [ ]:
# Show the first 5 rows of the refined dataframe
column_names = list(refined_df).append('category')
updated_df = pd.DataFrame(refined_df, columns=column_names)
updated_df.head()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,predictions
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e,INFORMATION-TECHNOLOGY
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a,PUBLIC-RELATIONS
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0,HEALTHCARE
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783,ENGINEERING
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c,SALES


In [27]:
updated_df.to_csv('data/processed/jobposts.csv', index=False)

In [28]:
updated_df['category'].value_counts()

category
HEALTHCARE                3469
SALES                     3448
INFORMATION-TECHNOLOGY    3236
ENGINEERING               2691
CONSTRUCTION              1626
HR                         966
ACCOUNTANT                 925
CHEF                       917
ADVOCATE                   733
AUTOMOBILE                 707
FINANCE                    515
PUBLIC-RELATIONS           346
BUSINESS-DEVELOPMENT       310
AGRICULTURE                171
TEACHER                    166
AVIATION                   131
CONSULTANT                 110
DESIGNER                    99
BANKING                     91
DIGITAL-MEDIA               88
APPAREL                     83
FITNESS                     16
ARTS                         8
Name: count, dtype: int64

***

# Let's do a manual classification of job posts so we can check how well is AI doing and if the results are acceptable   

In [ ]:
import pandas as pd

In [ ]:
# Load the dataset
df = pd.read_csv('data/processed/jobposts.csv')

# Select the first 100 rows
subset = df.head(100)

# Create a new column to store user input
subset['manually_classified_category'] = ''

# Loop through each row and prompt the user for input
for i, row in subset.iterrows():
    print(f"\nJob #{i+1}")
    print("Title:", row['job_title'])
    print("Description:", row['job_description'])
    category = input("Enter job category: ")
    print("Actual ", row['predictions'])
    subset.at[i, 'manually_classified_category'] = category
